In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import os 

# Set dir depending on machine 
data_dir = os.getenv("DATADIR")

# Read in Step 4 Complete Catalog
l_df = pd.read_csv(data_dir + "0_SUMS_Catalogs/CompleteCatalog/Step4/lmc_photometry.csv")
s_df = pd.read_csv(data_dir + "0_SUMS_Catalogs/CompleteCatalog/Step4/smc_photometry.csv")

# Sort by RA
l_df = l_df.sort_values(by='ra').reset_index(drop=True)
s_df = s_df.sort_values(by='ra').reset_index(drop=True)

# Add ID Column 
l_df['SUMS_ID'] =  ['SUMS_'+ str(hex(i))[2:] for i in np.arange(100000, 100000 + l_df.shape[0])]
s_df['SUMS_ID'] =  ['SUMS_'+ str(hex(i))[2:] for i in np.arange(100000 +l_df.shape[0],100000 +l_df.shape[0]+s_df.shape[0])]


# Combine 
# df = pd.concat([l_df, s_df]).reset_index(drop=True)

for df,galaxy in zip([l_df,s_df],['LMC','SMC']):
  # Drop Unnamed cols 
  df = df.drop(columns=df.columns[df.columns.str.contains('Unnamed')])

  # Resort columns
  cols = [ # Basic info
          'SUMS_ID', 'ra', 'dec', 
          # UV Photometry
          'uvw2_mag', 'uvw2_mag_err', 'uvm2_mag', 'uvm2_mag_err',  'uvw1_mag', 'uvw1_mag_err',
          # Optical Photometry
          'U', 'e_U', 'B', 'e_B', 'V', 'e_V', 'I', 'e_I',
          # Standard Deviation 
          'uvw2_mag_std', 'uvm2_mag_std', 'uvw1_mag_std',
          # Flux Frac 
          'uvw2_flux_frac', 'uvm2_flux_frac', 'uvw1_flux_frac',
          # Residual Frac
          'uvw2_resid_frac',  'uvm2_resid_frac', 'uvw1_resid_frac',
          # Dist Moved
          'uvw2_dist_moved',  'uvm2_dist_moved', 'uvw1_dist_moved',
          # Dist to nearest neighbor 
          'uvw2_closest_min',  'uvm2_closest_min', 'uvw1_closest_min',
          # Num5 
          'uvw2_num5','uvm2_num5', 'uvw1_num5', 
          # Num2.5
          'uvw2_num2p5',  'uvm2_num2p5', 'uvw1_num2p5',
          # N observations 
          'uvw2_num_obs', 'uvm2_num_obs',  'uvw1_num_obs']
  df = df[cols]

  # Rename Columns
  df.columns = ['SUMS_ID', 'RA', 'Dec', 
                
                'UVW2', 'UVW2_err', 'UVM2', 'UVM2_err', 'UVW1', 'UVW1_err',

                'U', 'U_err', 'B', 'B_err', 'V', 'V_err', 'I', 'I_err',

                'UVW2_std', 'UVM2_std', 'UVW1_std',

                'UVW2_flux_frac', 'UVM2_flux_frac', 'UVW1_flux_frac',

                'UVW2_resid_frac', 'UVM2_resid_frac', 'UVW1_resid_frac',

                'UVW2_dist_moved', 'UVM2_dist_moved', 'UVW1_dist_moved',

                'UVW2_dist_neighbor', 'UVM2_dist_neighbor', 'UVW1_dist_neighbor',

                'UVW2_n5', 'UVM2_n5', 'UVW1_n5',

                'UVW2_n2p5', 'UVM2_n2p5', 'UVW1_n2p5',

                'UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs']

  # Report same SF as MCPS
  sig_fig_cols = ['UVW2', 'UVM2', 'UVW1','UVW2_err', 'UVM2_err', 'UVW1_err','UVW2_std', 'UVM2_std', 'UVW1_std',

                'UVW2_flux_frac', 'UVM2_flux_frac', 'UVW1_flux_frac',

                'UVW2_resid_frac', 'UVM2_resid_frac', 'UVW1_resid_frac',

                'UVW2_dist_moved', 'UVM2_dist_moved', 'UVW1_dist_moved',

                'UVW2_dist_neighbor', 'UVM2_dist_neighbor', 'UVW1_dist_neighbor']


  for col in sig_fig_cols:
    df[col] = df[col].round(2)

  print('Final Size:', df.shape[0])
  # Save final catalog 
  df = df.reset_index(drop=True)
  df.to_csv(data_dir + f'0_SUMS_Catalogs/FinalCatalogs/SUMS_UV_Catalog_{galaxy}.csv', index=False)

Final Size: 471504
Final Size: 263358


In [2]:
# Put into machine readable table for latex 

# Choose a few rows that have data and are more typical
# Only show some columns otherwise it's too wide
# Add a columns of 'vdots' to split 
df['\\hdots'] = ['\\hdots']*df.shape[0]
selection = df.loc[[1235,789,732138,734700],['SUMS_ID', 'RA', 'Dec', 
              
               'UVW2', 'UVW2_err', 'UVM2', 

                '\\hdots',

             'UVW1_n2p5',

              'UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs']]

# Format the floats
for col in [ 'UVW2', 'UVW2_err', 'UVM2']:
  selection[col] = selection[col].apply(lambda x: '{:.5g}'.format(x))
# Format the integers 
for col in ['UVW2_nobs', 'UVM2_nobs', 'UVW1_nobs','UVW1_n2p5']:
  selection[col] = selection[col].apply(lambda x: '{:.0f}'.format(x))


# Anything with underscores needs a \\ in front of it 
df['SUMS_ID'] = df['SUMS_ID'].apply(lambda x: x.replace('_','$\\_$'))
columns = selection.columns
columns = [col.replace('_','$\\_$') for col in columns]
selection.columns = columns


print(selection.to_latex(index=False))

KeyError: '[732138, 734700] not in index'